# Playground

In [61]:
import pandas as pd
import feather
import pycountry
import dash
from dash import dcc 
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

In [62]:
# load feather file from input directory
df = pd.read_feather('../input/df_2015')

In [63]:
def alpha2_to_alpha3(alpha_2):
    '''Converts ISO 2 country code to ISO 3 country code'''
    country = pycountry.countries.get(alpha_2=alpha_2)
    return country.alpha_3 if country else alpha_2

In [64]:
## Adds country name to dataframe
country_mapping = {country.alpha_3: country.name for country in pycountry.countries}

In [65]:
df['Country'] = df['Country'].apply(alpha2_to_alpha3)

In [66]:
df['Country_Name'] = df['Country'].map(country_mapping)

In [67]:
not_converted = df[df['Country'].str.len() == 2]['Country'].unique()
print(not_converted)

# Drop rows where 'Country' column contains 2-letter codes
df = df[df['Country'].str.len() != 2]

['ll' 'DD' 'EP']


In [68]:
# rename column earlist_year to year
df.rename(columns={'earliest_filing_year': 'Year'}, inplace=True)

In [69]:
# Retrieves alle ISO 3 country codes and country names from pycountry
countries_df = pd.DataFrame({
    'Country': [country.alpha_3 for country in pycountry.countries],
    'Country_Name': [country.name for country in pycountry.countries]
})

In [70]:
# create a DataFrame that includes all years in your original dataset
years_df = pd.DataFrame({'Year': df['Year'].unique()})

In [71]:
# create a DataFrame that includes every combination of country and year.
countries_years_df = pd.merge(
    countries_df.assign(key=0),
    years_df.assign(key=0),
    on='key'
).drop('key', axis=1)

In [73]:
# merge this countries_years_df DataFrame with original df 
df = pd.merge(countries_years_df, df, on=['Country', 'Country_Name', 'Year'], how='outer')


In [74]:
# Replace NaN values with 0
df['ff'] = df['ff'].fillna(0)
df['renewables'] = df['renewables'].fillna(0)


In [86]:
## Create a better color scale
color_scale_renewables = [
    (0.0, 'rgb(247,252,245)'),
    (0.05, 'rgb(229,245,224)'),
    (0.1, 'rgb(199,233,192)'),
    (0.2, 'rgb(161,217,155)'),
    (0.3, 'rgb(116,196,118)'),
    (0.4, 'rgb(65,171,93)'),
    (0.5, 'rgb(35,139,69)'),
    (0.6, 'rgb(0,109,44)'),
    (0.7, 'rgb(0,68,27)'),
    (0.8, 'rgb(0,68,27)'),
    (0.9, 'rgb(0,68,27)'),
    (1.0, 'rgb(0,68,27)')  # Dark green for all values above 150
]

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id="type-selection", 
        options=[
            {"label": "Fossil Fuel Energy Innovation", "value": 'ff'},
            {"label": "Renewable Energy Innovation", "value": 'renewables'}
        ],
        value='ff',
        clearable=False
    ),
    dcc.Graph(id="world-map", style={"height": "800px", "width": "100%"})  # Adjust the size here
])


@app.callback(
    Output("world-map", "figure"), 
    [Input("type-selection", "value")]
)
def update_world_map(energy_type):
    if energy_type == "ff":
        color_scale = "turbid"  # brownish color scale for fossil fuels
    else:
        color_scale = color_scale_renewables  # custom green color scale for renewables
    
    fig = px.choropleth(
        df, 
        locations="Country", 
        color=energy_type, 
        hover_name="Country_Name", 
        animation_frame="Year",
        projection="natural earth",
        color_continuous_scale=color_scale,
        labels={energy_type: 'Number of Patents'}
    )
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)
    print("The app is running at: http://localhost:8050")

The app is running at: http://localhost:8050
